# 5. Function fitting and Interpolation

In today's session, we will be using some of the LAI datasets we examined last week (masked by national boundaries) and doing some analysis on them.

- [5.1 Making 3D datasets and Movies](#5.1-Making-3D-datasets-and-Movies)
    First, we will examine how to improve our data reading function by extracting only the area we are interested in. This involves querying the 'country' mask to find its limits and passing this information through to the reader.

- [5.2 Interpolation](#5.2-Interpolation)
    Then we will look at methods to interpolate and smooth over gaps in datasets using various methods.

- [5.3 Function Fitting](#5.3-Function-fitting)
    Finally, we will look at fitting models to datasets, in this case a model describing LAI phenology.

## 5.1 Making 3D datasets and Movies

First though, we will briefly go over once more the work we did on downloading data (ussssing `wget`), generating 3D masked datasets, and making movies.

This time, we will concentrate more on generating functions that we can re-use for other purposes.

### 5.1.1 Downloading data

We can download data from the MODIS server. The URL for this is [http://e4ftl01.cr.usgs.gov//MODV6_Cmp_A/MOTA/MCD15A2H.006/](http://e4ftl01.cr.usgs.gov//MODV6_Cmp_A/MOTA/MCD15A2H.006/), and you can see that inside that URL, we have a folder per time step. Inside that folder, all the files for each of the land tiles are present. Currently, you need an username and password to access this folders, which you can [register for here](https://urs.earthdata.nasa.gov/). You should then be able to click on the links and see and download individual files.

Let's try to write some Python code to download individual files. The code will try to do the following:

1. Open the URL [http://e4ftl01.cr.usgs.gov//MODV6_Cmp_A/MOTA/MCD15A2H.006/2013.02.18](http://e4ftl01.cr.usgs.gov//MODV6_Cmp_A/MOTA/MCD15A2H.006/2013.02.18), and retrieve the HTML content using the `requests` package
2. Parse the content of the HTML to extract the URLs from all the available links using the BeautifulSoup package.
3. Check that the files have our tile of interest, as well as end in `.hdf`

Let's see how this works...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# The following two modules allow you to...
import bs4 # Parse HTML data
import requests     # Get data on the internet using an URL

the_url = 'http://e4ftl01.cr.usgs.gov//MODV6_Cmp_A/MOTA/MCD15A2H.006/'
tile = "h31v11"
dates = "2013.02.18" # Eg.

r = requests.get (the_url + dates)
if not r.ok:
    print "Problem!"
html_content = r.content

soup = bs4.BeautifulSoup(html_content)
for link in soup.findAll("a"):
    fname = link.get("href")
    if fname.find (tile) >= 0 and fname.split(".")[-1] == "hdf":
        print fname


### 5.1.2 Read multiple dates

The previous code is OK for a single date, but we can make it even more useful if we stick it in a function to download data for a particular date. 

We might want to check that the date is a valid date for the particular dataset that we're targetting....

In [ ]:
def get_modis_fname ( modis_product_url='http://e4ftl01.cr.usgs.gov//MODV6_Cmp_A/MOTA/MCD15A2H.006/',
                    tile="h31v11", date="2013.02.18"):
    """Get the URL for a MODIS Coll5 HDF file for a given tile and date.
    
    Parameters
    -----------
    
    modis_product_url: str
        A valid URL that points to the MODIS product
    tile: str
        A tile in MODIS parlance (e.g. "h17v03")
    date: str
        A date, in year.month.date
        
    Returns
    --------
    
    Either a full URL to download, or None if the e.g. 
    the date isn't available.
    """
    import bs4
    import requests
    
    r = requests.get ( modis_product_url + "/" + date )
    if not r.ok:
        # A problem was found. Warn the user and return None
        print "There was a problem obtaining the URL %s" % ( the_url + date )
        return None
    html_content = r.content

    soup = bs4.BeautifulSoup(html_content)
    for link in soup.findAll("a"):
        fname = link.get("href")
        if fname.find (tile) >= 0 and fname.split(".")[-1] == "hdf":
            return modis_product_url + "/" + date + "/" + fname 
                         # We assume we only have a single file fitting the 
                         # previous criteria
                
                
                
print get_modis_fname ( modis_product_url='http://e4ftl01.cr.usgs.gov//MODV6_Cmp_A/MOTA/MCD15A2H.006/',
                    tile="h31v11", date="2013.02.18")
print get_modis_fname ( modis_product_url='http://e4ftl01.cr.usgs.gov//MODV6_Cmp_A/MOTA/MCD15A2H.006/',
                    tile="h31v11", date="2013.02.19")

In order to retrieve the file, we need a username and password (see above), and we need to authenticate ourselves in order to download the data. The following code (taken from [get_modis](http://github.com/jgomezdans/get_modis) deals with the authentication, but also does some other useful stuff:

* Requests the file size from the remote server
* Works out how long it took to download the data
* Uses a *context manager* to open a file.

Perhaps the context manager is the weirdest concept here, but it's just a safe and convenient way of opening and closing a file. The following code
    
```python
    fp = open ("myfile.txt", 'w')
    fp.write ( x )
    fp.close()
```
    
can be written as

```python
    with open("myfile.txt", 'w') as fp:
        fp.write ( x )
```

The second option is preferred if you're doing more complicated stuff than just writing, as if something fails inside the context manager, `fp` will always be closed.

Note that we also use a context manager to authenticate ourselves against the MODIS server. The way the authentication works is through a redirection: you ask for the URL, and the server tells you to go somewhere else for authentication and then redirects you back to your HDF file.

In [ ]:
def get_modis_product ( url, username, password, out_dir="./"):
    """"
    Download a MODIS product from a given URL. This method requires an
    username and password, as well as an output directory (optional) to
    save the file. The method will authenticate against the server, get
    the data, and report some timing statistics.
    
    Parameters
    ------------
    
    url: str
        The URL. Must exist, otherwise, we raise an IOError exception
    username: str
        The useranme. I bet that was surprising.
    password: str
        The day of the week. Naaaah, the password!!!
    out_dir: str
        The output directory where the file will be downloaded.
        
    Returns
    --------
    Nothing, it reports some speed statistics on the command line.
    """
    import requests
    import os
    import time
    # The following is an HTTP session, denoted by `s`
    with requests.Session() as s:
        # Set up the authentication
        s.auth = (username, password)
        # Get the named URL from the user. This will 
        # result in an HTTP redirection for which we
        # need to harvest the url
        r1 = s.request('get', url)
        # The redirection is stored in the .url bit
        # We also open as a stream for downloading larger
        # files using an iterator.
        r = s.get(r1.url, stream=True)
        # Check whether it all worked
        if not r.ok:
            raise IOError("Can't start download... [%s]" % url)
        # Get the file size from the headers. File size is in bytes
        file_size = int(r.headers['content-length'])
        # Get the filename from the url
        fname = url.split("/")[-1]
        print "Starting download on %s(%d bytes) ..." % ( 
            os.path.join(out_dir, fname), file_size)
        # The next line stores the current time in seconds. It's our
        # start time
        tic = int(round(time.time() ))
        # Open the output file to write
        with open(os.path.join(out_dir, fname), 'wb') as fp:
            # This iterates over the remote file contents in
            # 64kbytes chunks. If the chunk is valid, we just
            # write to the output file
            for chunk in r.iter_content(chunk_size=65535):
                if chunk:
                    fp.write(chunk)
        # Time at the end
        toc = int(round(time.time() ))
        print "Done in %d s. Speed = %g bytes/second" % ( 
            (toc-tic), float(file_size)/ (toc-tic))
        
        
the_url = get_modis_fname ()    
username = "ProfLewis"
password = "GeogG1222016"
get_modis_product ( the_url, username, password)

### 5.1.3 Read Just The Data We Want

Last time, we generated a function to read MODIS LAI data.

We have now included such a function in the directory [`files/python`](files/python) called [`get_lai.py`](files/python/get_lai.py).

The only added sophistication is that when we call `ReadAsArray`, we give it the starting cols, rows, and number of cols and rows to read (e.g. `xsize=600,yoff=300,xoff=300,ysize=600`):

In [ ]:
import glob
filelist = glob.glob("data/MCD15A2.A2005*hdf")

# Now we have a list of filenames
# load read_lai
import sys
sys.path.insert(0,'python')

from get_lai import get_lai

help(get_lai)

In [ ]:
# e.g. for reading a single file:

lai_file0 = get_lai(filelist[20],ncol=600,mincol=300,minrow=400,nrow=800)
plt.imshow(lai_file0['Lai_1km'])

In [ ]:
print type(lai_file0)
print lai_file0.keys()

The function returns a dictionary with has keys `['Lai_1km', 'LaiStdDev_1km', 'FparLai_QC']`:

In [ ]:
print lai_file0['Lai_1km'].shape

Each of these datasets is of shape `(1200, 1200)`, but we have read only 600 (columns) and 800 (rows) in this case. Note that the numpy indexing is `(rows,cols)`.

We know how to create a mask from a vector dataset from thelast session:

In [ ]:
# have to make sure have access to gdal data files 
import os
if 'GDAL_DATA' not in os.environ:
    os.environ["GDAL_DATA"] = '/opt/anaconda/share/gdal'

from raster_mask import raster_mask

# make a raster mask
# from the layer IRELAND in world.shp
filename = filelist[0]
file_template = 'HDF4_EOS:EOS_GRID:"%s":MOD_Grid_MOD15A2:%s'
file_spec = file_template%(filename,'Lai_1km')
                           
mask = raster_mask(file_spec,
                   target_vector_file = "data/world.shp",\
                   attribute_filter = "NAME = 'IRELAND'")


plt.imshow(mask)
plt.colorbar()

In this case, the data we want is only a small section of the whole spatial dataset.

It would be convenient to extract *only* the part we want.

We can use `numpy.where()` to help with this:

In [ ]:
# The mask is False for the area we want
rowpix,colpix = np.where(mask == False)

print rowpix,colpix

`rowpix` and `colpix` are lists of pixel coordinates where the condition we specified is `True` (i.e. where `mask` is `False`).

If we wanted to find the bounds of this area, we simply need to know the minimum and maximum column and row in these lists:

In [ ]:
mincol,maxcol = min(colpix),max(colpix)
minrow,maxrow = min(rowpix),max(rowpix)

# think about why the + 1 here!!!
# what if maxcol and mincol were the same?
ncol = maxcol - mincol + 1
nrow = maxrow - minrow + 1

print minrow,mincol,nrow,ncol

We could use this information to extract *only* the area we want when we read the data:

In [ ]:
lai_file0 = get_lai(filelist[20],\
                    ncol=ncol,nrow=nrow,mincol=mincol,minrow=minrow)

plt.imshow(lai_file0['Lai_1km'],interpolation='none')

Now, lets extract this portion of the mask:

In [ ]:
small_mask = mask[minrow:minrow+nrow,mincol:mincol+ncol]

plt.imshow(small_mask,interpolation='none')

And combine the country mask with the small dataset:

As a recap, we can use the function `raster_mask` that we gave you last time to develop a raster mask (!) from an ESRI shapefile (`data/world.shp` here).

We can then combine this mask with the QC-derived mask in the LAI dataset.

The LAI mask (that will be `lai.mask` in the code below) is `False` for good data, as is the coutry mask.

To combine them, we want some operator `X` for which:

`True  X True  == True`  
`True  X False == True`  
`False X True  == True`  
`False X False == False`  

The operator to use then is an *or*, here, a bitwise or, `|`.


In [ ]:
import numpy.ma as ma
lai_file0 = get_lai(filelist[20],
                    ncol=ncol,nrow=nrow,mincol=mincol,minrow=minrow)

layer = 'Lai_1km'
lai = lai_file0[layer]
small_mask = mask[minrow:minrow+nrow,mincol:mincol+ncol]

# combined mask
new_mask = small_mask | lai.mask

plt.figure(figsize=(7,7))
plt.imshow(new_mask,interpolation='none')

lai = ma.array(lai,mask=new_mask)

plt.figure(figsize=(7,7))
plt.imshow(lai,interpolation='none')


We should be used to writing loops around such functions.

In this case, we read *all* of the files in `filelist` and put the data into the dictionary called `lai` here.

Because there are multiple layers in the datasets, we loop over layer and append to each list indiviually:

In [ ]:
# load 'em all ...

# for United Kingdom here

import numpy.ma as ma
from raster_mask import raster_mask

country = 'UNITED KINGDOM'

# make a raster mask
# from the layer UNITED KINGDOM in world.shp
filename = filelist[0]
file_template = 'HDF4_EOS:EOS_GRID:"%s":MOD_Grid_MOD15A2:%s'
file_spec = file_template%(filename,'Lai_1km')
                           
mask = raster_mask(file_spec,
                   target_vector_file = "data/world.shp",
                   attribute_filter = "NAME = '%s'"%country)
# extract just the area we want
# by getting the min/max rows/cols
# of the data mask
# The mask is False for the area we want
rowpix,colpix = np.where(mask == False)
mincol,maxcol = min(colpix),max(colpix)
minrow,maxrow = min(rowpix),max(rowpix)
ncol = maxcol - mincol + 1
nrow = maxrow - minrow + 1
# and make a small mask
small_mask = mask[minrow:minrow+nrow,mincol:mincol+ncol]


# data_fields with empty lists
data_fields = {'LaiStdDev_1km':[],'Lai_1km':[]}

# make a dictionary and put the filenames in it
# along with the mask and min/max info
lai = {'filenames':np.sort(filelist),
       'minrow':minrow,'mincol':mincol,
       'mask':small_mask}

# combine the dictionaries
lai.update(data_fields)

# loop over each filename
for f in np.sort(lai['filenames']):
    this_lai = get_lai(f,
                       mincol=mincol,ncol=ncol,
                       minrow=minrow,nrow=nrow)
    for layer in data_fields.keys():
        # apply the mask
        new_mask = this_lai[layer].mask | small_mask
        this_lai[layer] = ma.array(this_lai[layer],mask=new_mask)
        lai[layer].append(this_lai[layer])
        

In [ ]:
# have a look at one of these

i = 20

# just see what the shape is ...
print lai['Lai_1km'][i].shape

root = 'images/lai_uk'

cmap = plt.cm.Greens

f = lai['filenames'][i]
fig = plt.figure(figsize=(7,7))
# get some info from filename
file_id = f.split('/')[-1].split('.')[-5][1:]
print file_id
plt.imshow(lai['Lai_1km'][i],cmap=cmap,interpolation='none',vmax=4.,vmin=0.0)
# plot a jpg
plt.title(file_id)
plt.colorbar()
plt.savefig('images/lai_uk_%s.jpg'%file_id)

In [ ]:
# thats quite good, so put as a function:
import numpy.ma as ma
import numpy as np
import sys
sys.path.insert(0,'python')
from get_lai import get_lai
from raster_mask import raster_mask


def read_lai(filelist,country=None):
    '''
    Read MODIS LAI data from a set of files
    in the list filelist. Data assumed to be in
    directory datadir.
    
    Parameters:
    filelist : list of LAI files
    
    Options:
    country  : country name (in data/world.shp)
    
    Returns:
    lai dictionary
    '''
    if country:
        # make a raster mask
        # from the layer UNITED KINGDOM in world.shp
        file_template = 'HDF4_EOS:EOS_GRID:"%s":MOD_Grid_MOD15A2:%s'
        file_spec = file_template%(filelist[0],'Lai_1km')
                                   
        mask = raster_mask(file_spec,\
                           target_vector_file = "data/world.shp",\
                           attribute_filter = "NAME = '%s'"%country)
        # extract just the area we want
        # by getting the min/max rows/cols
        # of the data mask
        # The mask is False for the area we want
        rowpix,colpix = np.where(mask == False)
        mincol,maxcol = min(colpix),max(colpix)
        minrow,maxrow = min(rowpix),max(rowpix)
        ncol = maxcol - mincol + 1
        nrow = maxrow - minrow + 1
        # and make a small mask
        small_mask = mask[minrow:minrow+nrow,mincol:mincol+ncol]
    else:
        # no country
        mincol = 0
        maxcol = 0
        ncol = None
        nrow = None

    # data_fields with empty lists
    data_fields = {'LaiStdDev_1km':[],'Lai_1km':[]}
    
    # make a dictionary and put the filenames in it
    # along with the mask and min/max info
    lai = {'filenames':np.sort(filelist),
           'minrow':minrow,'mincol':mincol,
           'mask':small_mask}
    
    # combine the dictionaries
    lai.update(data_fields)
    
    # loop over each filename
    for f in np.sort(lai['filenames']):
        this_lai = get_lai(f,
                           mincol=mincol,ncol=ncol,
                           minrow=minrow,nrow=nrow)
        for layer in data_fields.keys():
            # apply the mask
            if country:
                new_mask = this_lai[layer].mask | small_mask
                this_lai[layer] = ma.array(this_lai[layer],mask=new_mask)
            lai[layer].append(this_lai[layer])   
    for layer in data_fields.keys():
        lai[layer] = ma.array(lai[layer])
            
    return lai

In [ ]:
# test this ... the one in the file
# does a cutout of the data area as well
# which will keep the memory
# requirements down
#from get_lai import read_lai

lai = read_lai(filelist,country='IRELAND')

# have a look at one of these

i = 20

# just see what the shape is ...
print lai['Lai_1km'][i].shape

root = 'images/lai_eire'

cmap = plt.cm.Greens

f = lai['filenames'][i]
fig = plt.figure(figsize=(7,7))
# get some info from filename
file_id = f.split('/')[-1].split('.')[-5][1:]
print file_id
plt.imshow(lai['Lai_1km'][i],cmap=cmap,interpolation='none',vmax=4.,vmin=0.0)
# plot a jpg
plt.title(file_id)
plt.colorbar()
plt.savefig('%s_%s.jpg'%(root,file_id))

In [ ]:
# make a movie

import pylab as plt
import os

# just see what the shape is ...
print lai['Lai_1km'].shape

root = 'images/lai_country_eire'

cmap = plt.cm.Greens

for i,f in enumerate(lai['filenames']):
    fig = plt.figure(figsize=(7,7))
    # get some info from filename
    file_id = f.split('/')[-1].split('.')[-5][1:]
    print file_id
    plt.imshow(lai['Lai_1km'][i],cmap=cmap,interpolation='none',vmax=4.,vmin=0.0)
    # plot a jpg
    plt.title(file_id)
    plt.colorbar()
    plt.savefig('%s_%s.jpg'%(root,file_id))
    plt.close(fig)
    
cmd = 'convert -delay 100 -loop 0 {0}_*.jpg {0}_movie.gif'.format(root)
os.system(cmd)

![](files/images/lai_country_eire_movie.gif)

In [ ]:
# The movie making works, so pack that into a function

import pylab as plt
import os

root = 'images/lai_eire'

def make_movie(lai,root,layer='Lai_1km',vmax=4.,vmin=0.,do_plot=False):
    '''
    Make an animated gif from MODIS LAI data in
    dictionary 'lai'.
    
    Parameters:
    lai    : data dictionary
    root   : root file /directory name of frames and movie
    
    layer  : data layer to plot 
    vmax   : max value for plotting
    vmin   : min value for plotting
    do_plot: set True if you want the individual plots
             to display
    
    Returns:
    movie name    
    
    '''
    cmap = plt.cm.Greens
    
    for i,f in enumerate(lai['filenames']):
        fig = plt.figure(figsize=(7,7))
        # get some info from filename
        file_id = f.split('/')[-1].split('.')[-5][1:]
        print file_id
        plt.imshow(lai[layer][i],cmap=cmap,interpolation='none',\
                   vmax=vmax,vmin=vmin)
        # plot a jpg
        plt.title(file_id)
        plt.colorbar()
        plt.savefig('%s_%s.jpg'%(root,file_id))
        if not do_plot:
            plt.close(fig)
        
    cmd = 'convert -delay 100 -loop 0 {0}_*.jpg {0}_movie.gif'.format(root)
    os.system(cmd)
    return '{0}_movie.gif'.format(root)

In [ ]:
# test it

lai_uk = read_lai(filelist,country='UNITED KINGDOM')
root = 'images/lai_UK'
movie = make_movie(lai_uk,root)
print movie

![](files/images/lai_UK_movie.gif)

## 5.2 Interpolation

### 5.2.1 Univariate interpolation

So, we can load the data we want from multiple MODIS hdf files that we have downloaded from the NASA server into a 3D masked numpy array, with a country boundary mask (projected int the raster data coordinate system) from a vector dataset.

Let's start to explore the data then.

You should have an array of LAI for Ireland:

In [ ]:
type(lai['Lai_1km'])

Let's plot the LAI for some given pixels.

First, we might like to identify which pixels actually have any data.

A convenient function for this would be `np.where` that returns the indices of items that are `True`.

Since the data mask is `False` for good data, we take the complement `~` so that good data are `True:

In [ ]:
data = lai['Lai_1km']
np.where(~data.mask)

An example good pixel this is (3,329,145). Let's look at this for all time periods:

In [ ]:
data = lai['Lai_1km']

r = 329
c = 83

pixel = data[:,r,c]

# plot red stars at the data points
plt.plot(np.arange(len(pixel))*8,pixel,'r*')
# plot a black (k) dashed line (--)
plt.plot(np.arange(len(pixel))*8,pixel,'k--')
plt.xlabel('doy')
plt.ylabel('LAI')
plt.title('pixel %03d %03d'%(r,c))

The data follow the trend of what we might expect for LAI development, but they are clearly a little noisy.

We also have access to uncertainty information (standard deviation):

In [ ]:
# copy the data in case we change it any

data = lai['Lai_1km'].copy()
sd   = lai['LaiStdDev_1km'].copy()

r = 329
c = 83

pixel    = data[:,r,c]
pixel_sd =   sd[:,r,c]

x = np.arange(len(pixel))*8

# plot red stars at the data points
plt.plot(x,pixel,'r*')
# plot a black (k) dashed line (--)
plt.plot(x,pixel,'k--')
# plot error bars:
# 1.96 because that is the 95% confidence interval
plt.errorbar(x,pixel,yerr=pixel_sd*1.96)
plt.xlabel('doy')
plt.ylabel('LAI')
plt.title('pixel %03d %03d'%(r,c))

We would generally expect LAI to be quite smoothly varying over time. Visualising the data with 95% confidence intervals is quite useful as we can now 'imagine' some smooth line that would generally go within these bounds.

Some of the uncertainty estimates are really rather small though, which are probably not reliable.

Let's inflate them:



In [ ]:

data = lai['Lai_1km'].copy()
sd   = lai['LaiStdDev_1km'].copy()

r = 329
c = 83

pixel    = data[:,r,c]
pixel_sd =   sd[:,r,c]
# threshold
thresh = 0.25
pixel_sd[pixel_sd<thresh] = thresh

x = np.arange(len(pixel))*8

# plot red stars at the data points
plt.plot(x,pixel,'r*')
# plot a black (k) dashed line (--)
plt.plot(x,pixel,'k--')
# plot error bars:
# 1.96 because that is the 95% confidence interval
plt.errorbar(x,pixel,yerr=pixel_sd*1.96)
plt.xlabel('doy')
plt.ylabel('LAI')
plt.title('pixel %03d %03d'%(r,c))

This is perhaps a bit more realistic ...

The data now have some missing values (data gaps) and, as we have noted, are a little noisy.

A Python module we can use for many scientific functions is [`scipy`](http://docs.scipy.org/doc/scipy), in particular here, the [`scipy` interpolation functions](http://docs.scipy.org/doc/scipy/reference/interpolate.html).

We need to make a careful choice of the interpolation functions.

We might, in many circumstances simply want something that interpolates between data points, i.e. that goes through the data points that we have.

Many interpolators will not provide extrapolation, so in the example above we could not get an estimate of LAI prior to the first sample and after the last.

The best way to deal with that would be to have multiple years of data.

Instead here, we will repeat the dataset three times to mimic this:


In [ ]:
from scipy import interpolate

pixel = data[:,r,c]

# original x,y
y_ = pixel
x_ = (np.arange(len(y_))*8.+1)[~pixel.mask]
y_ = y_[~pixel.mask]

# extend: using np.tile() to repeat data
y_extend = np.tile(y_,3)
# extend: using vstack to stack 3 different arrays
x_extend = np.hstack((x_-46*8,x_,x_+46*8))

In [ ]:
# plot the extended dataset
plt.figure(figsize=(12,3))
plt.plot(x_extend,y_extend,'b')
plt.plot(x_,y_,'k+')
plt.plot([0.,0.],[0.,2.5],'r')
plt.plot([365.,365.],[0.,2.5],'r')
plt.xlim(-356,2*365)
plt.xlabel('day of year')
plt.ylabel('LAI')

In [ ]:
# define xnew at 1 day interval
xnew = np.arange(1.,366.)

# linear interpolation
f = interpolate.interp1d(x_extend,y_extend,kind='linear')
ynew = f(xnew)

In [ ]:
plt.plot(xnew,ynew)
plt.plot(x_,y_,'r+')
plt.xlim(1,366)

In [ ]:
# cubic interpolation
f = interpolate.interp1d(x_extend,y_extend,kind='cubic')
ynew = f(xnew)
plt.plot(xnew,ynew)
plt.plot(x_,y_,'r+')
plt.xlim(1,366)

In [ ]:
# nearest neighbour interpolation
f = interpolate.interp1d(x_extend,y_extend,kind='nearest')
ynew = f(xnew)
plt.plot(xnew,ynew)
plt.plot(x_,y_,'r+')
plt.xlim(1,366)

Depending on the problem you are trying to solve, different interpolation schemes will be appropriate. For categorical data (e.g. 'snow', coded as 1 and 'no snow' coded as 1), for instance, a nearest neighbour interpolation might be a good idea.

### 5.2.2 Smoothing

One issue with the schemes above is that they go exactly through the data points, but a more realistic description of the data might be one that incorporated the uncertainty information we have. Visually, this is quite easy to imagine, but how can we implement such ideas?

One way of thinking about this is to think about other sources of information that we might bring to bear on the problem. One such would be that we expect the function to be 'quite smooth'. This allows us to consider applying smoothness as an additional constraint to the solution.

Many such problems can be phrased as convolution operations.

Convolution is a form of digital filtering that combines two sequences of numbers $y$ and $w$ to give a third, the result $z$ that is a filtered version of $y$, where for each element $j$ of $y$:

$$
  z_j = \sum_{i=-n}^{i=n}{w_i y_{j+i}}
$$

where $n$ is the half width of the filter $w$. For a smoothing filter, the elements of this will sum to 1 (so that the magnitude of $y$ is not changed).

To illustrate this in Python:



In [ ]:
# a simple box smoothing filter
# filter width 11
w = np.ones(11)
# normalise
w = w/w.sum()
# half width
n = len(w)/2

# Take the linear interpolation of the LAI above as the signal 
# linear interpolation
x = xnew
f = interpolate.interp1d(x_extend,y_extend,kind='linear')
y = f(x)

# where we will put the result
z = np.zeros_like(y)

# This is a straight implementation of the
# equation above
for j in xrange(n,len(y)-n):
    for i in xrange(-n,n+1):
        z[j] += w[n+i] * y[j+i]

In [ ]:
plt.plot(x,y,'k--',label='y')
plt.plot(x,z,'r',label='z')
plt.xlim(x[0],x[-1])
plt.legend(loc='best')
plt.title('smoothing with filter width %d'%len(w))

As we suggested, the result of convolving $y$ with the filter $w$ (of width 31 here) is $z$, a smoothed version of $y$.

You might notice that the filter is only applied once we are `n` samples into the signal, so we get 'edge effects'. There are various ways of dealing with edge effects, such as repeating the signal (as we did above, for much the same reason), reflecting the signal, or assuming the signal to be some constant value (e.g. 0) outside of its defined domain.

If we make the filter wider (width 31 now):

In [ ]:
# a simple box smoothing filter
# filter width 31
w = np.ones(31)
# normalise
w = w/w.sum()
# half width
n = len(w)/2

# Take the linear interpolation of the LAI above as the signal 
# linear interpolation
x = xnew
f = interpolate.interp1d(x_extend,y_extend,kind='linear')
y = f(x)

# where we will put the result
z = np.zeros_like(y)

# This is a straight implementation of the
# equation above
for j in xrange(n,len(y)-n):
    for i in xrange(-n,n+1):
        z[j] += w[n+i] * y[j+i]
        
plt.plot(x,y,'k--',label='y')
plt.plot(x,z,'r',label='z')
plt.xlim(x[0],x[-1])
plt.legend(loc='best')
plt.title('smoothing with filter width %d'%len(w))

Then the signal is 'more' smoothed. 

There are *many* filters implemented in [`scipy.signal`](http://docs.scipy.org/doc/scipy/reference/signal.html) that you should look over.

A very commonly used smoothing filter is the [Savitsky-Golay](http://en.wikipedia.org/wiki/Savitzky–Golay_filter_for_smoothing_and_differentiation) filter for which you define the window size and filter order.

As with most filters, the filter width controls the degree of smoothing (see examples above). The filter order (related to polynomial order) in essence controls the shape of the filter and defines the 'peakiness' of the response.

In [ ]:
import sys
sys.path.insert(0,'python')
# see http://wiki.scipy.org/Cookbook/SavitzkyGolay
from savitzky_golay import *

window_size = 31
order = 1

# Take the linear interpolation of the LAI above as the signal 
# linear interpolation
x = xnew
f = interpolate.interp1d(x_extend,y_extend,kind='linear')
y = f(x)

z = savitzky_golay(y,window_size,order)

plt.plot(x,y,'k--',label='y')
plt.plot(x,z,'r',label='z')
plt.xlim(x[0],x[-1])
plt.legend(loc='best')
plt.title('smoothing with filter width %d order %.2f'%(window_size,order))

In [ ]:
import sys
sys.path.insert(0,'python')
# see http://wiki.scipy.org/Cookbook/SavitzkyGolay
from savitzky_golay import *

window_size = 61
order = 2

# Take the linear interpolation of the LAI above as the signal 
# linear interpolation
x = xnew
f = interpolate.interp1d(x_extend,y_extend,kind='linear')
y = f(x)

z = savitzky_golay(y,window_size,order)

plt.plot(x,y,'k--',label='y')
plt.plot(x,z,'r',label='z')
plt.xlim(x[0],x[-1])
plt.legend(loc='best')
plt.title('smoothing with filter width %d order %.2f'%(window_size,order))

If the samples $y$ have uncertainty (standard deviation $\sigma_j$ for sample $j$) associated with them, we can incorporate this into smoothing, although many of the methods in `scipy` and `numpy` do not directly allow for this.

Instead, we call an optimal interpolation scheme (a regulariser) here that achieves this. This also has the advantage of giving an estimate of uncertainty for the smoothed samples.

In this case, the parameters are: `order` (as above, but only integer in this implementation) and `wsd` which is an estimate of the variation (standard deviation) in the signal that control smoothness.

In [ ]:
tile = 'h17v03'
year = '2005'

# specify the file with the urls in
ifile= 'data/modis_lai_%s_%s.txt'%(tile,year)

fp = open(ifile)
filelist = [url.split('/')[-1].strip() for url in fp.readlines()]
fp.close()
import sys
sys.path.insert(0,'python')

from get_lai import *

try:
    data = lai['Lai_1km']
    sd = lai['LaiStdDev_1km']
except:
    lai = read_lai(filelist,country='IRELAND')
    data = lai['Lai_1km']
    sd = lai['LaiStdDev_1km']
    
thresh = 0.25
sd[sd<thresh] = thresh

r = 472
c = 84
from smoothn import *

# this is about the right amount of smoothing here
gamma = 5.

pixel = data[:,r,c]
pixel_sd =   sd[:,r,c]

x = np.arange(46)*8+1

order = 2
z = smoothn(pixel,s=gamma,sd=pixel_sd,smoothOrder=2.0)[0]

# plot
plt.plot(x,pixel,'k*',label='y')
plt.errorbar(x,pixel,pixel_sd*1.96)
plt.plot(x,z,'r',label='z')
# lower and upper bounds of 95% CI

plt.xlim(1,366)
plt.ylim(0.,2.5)
plt.legend(loc='best')

In [ ]:
# test it on a new pixel

r = 472
c = 86

gamma = 5

pixel = data[:,r,c]
pixel_sd =   sd[:,r,c]

x = np.arange(46)*8+1

order = 2
z = smoothn(pixel,s=gamma,sd=pixel_sd,smoothOrder=2.0)[0]

# plot
plt.plot(x,pixel,'k*',label='y')
plt.errorbar(x,pixel,pixel_sd*1.96)
plt.plot(x,z,'r',label='z')

plt.xlim(1,366)
plt.legend(loc='best')
z.ndim

In [ ]:
# and test it on a new pixel

r = 472
c = 84

#r = 9
#c = 277
gamma = 5.

pixel = data[:,r,c]
pixel_sd =   sd[:,r,c]

x = np.arange(46)*8+1

order = 2
# solve for gamma - degree of smoothness 
zz = smoothn(pixel,sd=pixel_sd,smoothOrder=2.0)
z = zz[0]
print zz[1],zz[2]

gamma = zz[1]

# plot
plt.plot(x,pixel,'k*',label='y')
plt.errorbar(x,pixel,pixel_sd*1.96)
plt.plot(x,z,'r',label='z')

plt.xlim(1,366)
plt.legend(loc='best')

To apply this approach to our 3D dataset, we could simply loop over all pixels.

Note that *any* per-pixel processing will be slow ... but this is quite a fast smoothing method, so is feasible here.

In [ ]:
# we have put in an axis control to smoothn
# here so it will only smooth over doy
# This will take a few minutes to process
# we switch on verbose mode to get some feedback
# on progress

# make a mask of pixels where there is at least 1 sample
# over the time period
mask = (data.mask.sum(axis=0) == 0)
mask = np.array([mask]*data.shape[0])

z = smoothn(data,s=5.0,sd=sd,smoothOrder=2.0,axis=0,TolZ=0.05,verbose=True)[0]
z = ma.array(z,mask=mask)

In [ ]:
plt.figure(figsize=(9,9))
plt.imshow(z[20],interpolation='none',vmax=6)
plt.colorbar()

In [ ]:
# similarly, take frame 20
# and smooth that

ZZ = smoothn(z[20],smoothOrder=2.)
# self-calibrated smoothness term
s = ZZ[1]
print 's =',s
Z = ZZ[0]
plt.figure(figsize=(9,9))
plt.imshow(Z,interpolation='none',vmax=6)
plt.colorbar()

In [ ]:
# similarly, take frame 20
# and smooth that

ZZ = smoothn(z,s=s,smoothOrder=2.,axis=(1,2),verbose=True)

Z = ZZ[0]
plt.figure(figsize=(9,9))
plt.imshow(Z[30],interpolation='none',vmax=6)
plt.colorbar()

In [ ]:
x = np.arange(46)*8+1.
try:
    plt.plot(x,np.mean(Z,axis=(1,2)))
    plt.plot(x,np.min(Z,axis=(1,2)),'r--')
    plt.plot(x,np.max(Z,axis=(1,2)),'r--')
except:
    plt.plot(x,np.mean(Z,axis=2).mean(axis=1))
    plt.plot(x,np.min(Z,axis=2).min(axis=1),'r--')
    plt.plot(x,np.max(Z,axis=2).max(axis=1),'r--')
    
plt.title('LAI variation of Eire')

In [ ]:
# or doing this pixel by pixel ...
# which is slower than using axis

order = 2

# pixels that have some data
mask = (~data.mask).sum(axis=0)

odata = np.zeros((46,) + mask.shape)

rows,cols = np.where(mask>0)

len_x = len(rows)
order = 2
gamma = 5.

for i in xrange(len_x):
    r,c = rows[i],cols[i]
    # progress bar
    if i%(len_x/20) == 0:
        print '... %4.2f percent'%(i*100./float(len_x))
    pixel    = data[:,r,c]
    pixel_sd = sd[:,r,c]

    zz = smoothn(pixel,s=gamma,sd=pixel_sd,smoothOrder=order,TolZ=0.05)
    odata[:,rows[i],cols[i]] = zz[0]


In [ ]:
import pylab as plt
import os

root = 'images/lai_eire_colourZ'

for i,f in enumerate(lai['filenames']):
    fig = plt.figure(figsize=(7,7))
    # get some info from filename
    file_id = f.split('/')[-1].split('.')[-5][1:]
    print file_id
    plt.imshow(Z[i],interpolation='none',vmax=6.,vmin=0.0)
    # plot a jpg
    plt.title(file_id)
    plt.colorbar()
    plt.savefig('%s_%s.jpg'%(root,file_id))
    plt.close(fig)

In [ ]:
cmd = 'convert -delay 100 -loop 0 {0}_*.jpg {0}_movie2.gif'.format(root)
os.system(cmd)

![](files/images/lai_eire_colourZ_movie2.gif)

### 5.3 Function fitting

Sometimes, instead of applying some arbitrary smoothing function to data, we want to extract particular infromation from the time series.

One way to approach this is to fit some function to the time series at each location.

Let us suppose that we wish to characterise the phenology of vegetation in Ireland.

![](http://www2.geog.ucl.ac.uk/~plewis/geogg124/_images/zhang1.png)

One way we could do this would be to look in the lai data for the most rapid changes.

Another would be to explicitly fit some mathematical function to the LAI data that would would expect to descrive typical LAI trajectories.

One example of such a function is the double logistic. A logistic function is:

$$
 \hat{y} = p_0 - p_1 \left( \frac{1}{1 + e^{p_2 (t - p_3)}} + \frac{1}{1 + e^{p_4 (t - p_5)}} -1\right)
$$




We can give a function for a double logistic:

In [ ]:
def dbl_logistic_model ( p, t ):
        """A double logistic model, as in Sobrino and Juliean, 
        or Zhang et al"""
        return p[0] - p[1]* ( 1./(1+np.exp(p[2]*(t-p[3]))) + \
                              1./(1+np.exp(-p[4]*(t-p[5])))  - 1 )
   

In [ ]:
tile = 'h17v03'
year = '2005'

# specify the file with the urls in
ifile= 'data/modis_lai_%s_%s.txt'%(tile,year)

fp = open(ifile)
filelist = [url.split('/')[-1].strip() for url in fp.readlines()]
fp.close()
import sys
sys.path.insert(0,'python')

from get_lai import *

try:
    data = lai['Lai_1km']
    sd = lai['LaiStdDev_1km']
except:
    lai = read_lai(filelist,country='IRELAND')
    data = lai['Lai_1km']
    sd = lai['LaiStdDev_1km']
    
thresh = 0.25
sd[sd<thresh] = thresh

# test pixel
r = 472
c = 84


y = data[:,r,c]
mask = ~y.mask
y = np.array(y[mask])
x = (np.arange(46)*8+1.)[mask]
unc = np.array(sd[:,r,c][mask])

And see what this looks like:

In [ ]:
# define x (time)
x_full = np.arange(1,366) 

# some default values for the parameters
p = np.zeros(6)

# some stats on y
ysd = np.std(y)
ymean = np.mean(y)

# some rough guesses at the parameters

p[0] = ymean - 1.151*ysd;   # minimum  (1.151 is 75% CI)
p[1] = 2*1.151*ysd          # range
p[2] = 0.19                 # related to up slope
p[3] = 120                  # midpoint of up slope
p[4] = 0.13                 # related to down slope
p[5] = 220                  # midpoint of down slope

y_hat = dbl_logistic_model(p,x_full)

plt.clf()
plt.plot(x_full,y_hat)
plt.plot(x,y,'*')
plt.errorbar(x,y,unc*1.96)

We could manually 'tweak' the parameters until we got a better 'fit' to the observations.

First though, let's define a measure of 'fit':

$$
Z_i = \frac{\hat{y}_i - y_i}{\sigma_i}
$$

$$
Z^2 = \sum_i{Z_i^2} =  \sum_i{\left( \frac{\hat{y}_i - y_i}{\sigma_i} \right)^2}
$$

and implement this as a mismatch function where we have data points:

In [ ]:
def mismatch_function(p, x, y, unc):
    y_hat = dbl_logistic_model(p, x)
    diff = (y_hat - y)/unc
    return diff


Z = mismatch_function(p,x,y,unc)

plt.plot([1,365.],[0,0.],'k-')
plt.xlim(0,365)
plt.plot(x,Z,'*')


print 'Z^2 =',(Z**2).sum()

Now lets change p a bit:

In [ ]:
p[0] = ymean - 1.151*ysd;   # minimum  (1.151 is 75% CI)
p[1] = 2*1.151*ysd          # range
p[2] = 0.19                 # related to up slope
p[3] = 140                  # midpoint of up slope
p[4] = 0.13                 # related to down slope
p[5] = 220                  # midpoint of down slope

Z = mismatch_function(p,x,y,unc)

plt.plot([1,365.],[0,0.],'k-')
plt.xlim(0,365)
plt.plot(x,Z,'*')


print 'Z^2 =',(Z**2).sum()

We have made the mismatch go down a little ...

Clearly it would be tedious (and impractical) to do a lot of such tweaking, so we can use methods that seek the minimum of some function.

One such method is implemented in `scipy.optimize.leastsq`:

In [ ]:
from scipy import optimize

# initial estimate is in p
print 'initial parameters:',p[0],p[1],p[2],p[3],p[4],p[5]

# set some bounds for the parameters
bound = np.array([(0.,10.),(0.,10.),(0.01,1.),\
                  (50.,300.),(0.01,1.),(50.,300.)])


# test pixel
r = 472
c = 84


y = data[:,r,c]
mask = ~y.mask
y = np.array(y[mask])
x = (np.arange(46)*8+1.)[mask]
unc = np.array(sd[:,r,c][mask])

# define function to give Z^2

def sse(p,x,y,unc):
    '''Sum of squared error'''
    # penalise p[3] > p[5]
    err = np.max([0.,(p[3] - p[5])])*1e4
    return (mismatch_function(p,x,y,unc)**2).sum()+err

# we pass the function:
#
# sse               : the name of the function we wrote to give 
#                     sum of squares of Z_i
# p                 : an initial estimate of the parameters
# args=(x,y,unc)    : the other information (other than p) that
#                     mismatch_function needs
# approx_grad       : if we dont have a function for the gradien
#                     we have to get the solver to approximate it
#                     which takes time ... see if you can work out
#                     d_sse / dp and use that to speed this up!

psolve = optimize.fmin_l_bfgs_b(sse,p,approx_grad=True,iprint=-1,\
                                args=(x,y,unc),bounds=bound)

print psolve[1]
pp = psolve[0]
plt.plot(x,y,'*')
plt.errorbar(x,y,unc*1.96)
y_hat = dbl_logistic_model(pp,x_full)
plt.plot(x_full,y_hat)

print 'solved parameters: ',pp[0],pp[1],pp[2],pp[3],pp[4],pp[5]

# if we define the phenology as the parameter p[3]
# and the 'length' of the growing season:
print 'phenology',pp[3],pp[5]-pp[3]

In [ ]:
# and run over each pixel ... this will take some time

# pixels that have some data
mask = (~data.mask).sum(axis=0)

pdata = np.zeros((7,) + mask.shape)

rows,cols = np.where(mask>0)
len_x = len(rows)

# lets just do some random ones to start with
#rows = rows[::10]
#cols = cols[::10]

len_x = len(rows)


for i in xrange(len_x):
    r,c = rows[i],cols[i]
    # progress bar
    if i%(len_x/40) == 0:
        print '... %4.2f percent'%(i*100./float(len_x))
    
    y = data[:,r,c]
    mask = ~y.mask
    y = np.array(y[mask])
    x = (np.arange(46)*8+1.)[mask]
    unc = np.array(sd[:,r,c][mask])
    
    # need to get an initial estimate of the parameters
    
    # some stats on y
    ysd = np.std(y)
    ymean = np.mean(y)

    p[0] = ymean - 1.151*ysd;   # minimum  (1.151 is 75% CI)
    p[1] = 2*1.151*ysd          # range
    p[2] = 0.19                 # related to up slope
    p[3] = 140                  # midpoint of up slope
    p[4] = 0.13                 # related to down slope
    p[5] = 220                  # midpoint of down slope

    
    # set factr to quite large number (relative error in solution)
    # as it'll take too long otherwise
    psolve = optimize.fmin_l_bfgs_b(sse,p,approx_grad=True,iprint=-1,\
                                args=(x,y,unc),bounds=bound,factr=1e12)

    pdata[:-1,rows[i],cols[i]] = psolve[0]
    pdata[-1,rows[i],cols[i]] = psolve[1] # sse


In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(pdata[3],interpolation='none',vmin=137,vmax=141)
plt.title('green up doy')
plt.colorbar()

plt.figure(figsize=(10,10))
plt.imshow(pdata[5]-pdata[3],interpolation='none',vmin=74,vmax=84)
plt.title('season length')
plt.colorbar()

plt.figure(figsize=(10,10))
plt.imshow(pdata[0],interpolation='none',vmin=0.,vmax=6.)
plt.title('min LAI')
plt.colorbar()

plt.figure(figsize=(10,10))
plt.imshow(pdata[1]+pdata[0],interpolation='none',vmin=0.,vmax=6.)
plt.title('max LAI')
plt.colorbar()

plt.figure(figsize=(10,10))
plt.imshow(np.sqrt(pdata[-1]),interpolation='none',vmax=np.sqrt(500))
plt.title('RSSE')
plt.colorbar()

In [ ]:
# check a few pixels

c = 200

for r in xrange(200,400,25):
    y = data[:,r,c]
    mask = ~y.mask
    y = np.array(y[mask])
    x = (np.arange(46)*8+1.)[mask]
    unc = np.array(sd[:,r,c][mask])
    
    x_full = np.arange(1,366) 
    
    # some default values for the parameters
    pp = pdata[:-1,r,c]
    plt.figure(figsize=(7,7))
    plt.title('r %d c %d'%(r,c))
    plt.plot(x,y,'*')
    plt.errorbar(x,y,unc*1.96)
    y_hat = dbl_logistic_model(pp,x_full)
    plt.plot(x_full,y_hat)
    
    print 'solved parameters: ',pp[0],pp[1],pp[2],pp[3],pp[4],pp[5]
    
    # if we define the phenology as the parameter p[3]
    # and the 'length' of the growing season:
    print 'phenology',pp[3],pp[5]-pp[3]